# OpenAI APIs - Completions

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial covers the following popular APIs:

- `chat/completions`
- `completions`
- `batches`

Check out other tutorials to learn about [vision APIs](https://docs.sglang.ai/backend/openai_api_vision.html) for vision-language models and [embedding APIs](https://docs.sglang.ai/backend/openai_api_embeddings.html) for embedding models.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

INFO 03-19 07:17:37 __init__.py:190] Automatically detected platform cuda.


[2025-03-19 07:17:43] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=38520, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=1072561839, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, l

INFO 03-19 07:17:54 __init__.py:190] Automatically detected platform cuda.
INFO 03-19 07:17:54 __init__.py:190] Automatically detected platform cuda.


[2025-03-19 07:18:05 TP0] Init torch distributed begin.
[2025-03-19 07:18:06 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-03-19 07:18:06 TP0] Load weight begin. avail mem=78.81 GB


[2025-03-19 07:18:06 TP0] Using model weights format ['*.safetensors']


...

[2025-03-19 07:18:11 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-03-19 07:18:11] INFO:     Started server process [2894137]
[2025-03-19 07:18:11] INFO:     Waiting for application startup.
[2025-03-19 07:18:11] INFO:     Application startup complete.
[2025-03-19 07:18:11] INFO:     Uvicorn running on http://0.0.0.0:38520 (Press CTRL+C to quit)
[2025-03-19 07:18:11] INFO:     127.0.0.1:48222 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-19 07:18:12] INFO:     127.0.0.1:48232 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-19 07:18:12 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-19 07:18:15] INFO:     127.0.0.1:48242 - "POST /generate HTTP/1.1" 200 OK
[2025-03-19 07:18:15] The server is fired up and ready to roll!


Server started on http://localhost:38520


## Chat Completions

### Usage

The server fully implements the OpenAI API.
It will automatically apply the chat template specified in the Hugging Face tokenizer, if one is available.
You can also specify a custom chat template with `--chat-template` when launching the server.

In [2]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-03-19 07:18:16 TP0] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-19 07:18:17 TP0] Decode batch. #running-req: 1, #token: 76, token usage: 0.00, gen throughput (token/s): 6.54, #queue-req: 0, 
[2025-03-19 07:18:17] INFO:     127.0.0.1:48254 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

Here is an example of a detailed chat completion request:

In [3]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=128,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

print_highlight(response.choices[0].message.content)

[2025-03-19 07:18:17 TP0] Prefill batch. #new-seq: 1, #new-token: 51, #cached-token: 25, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-19 07:18:17 TP0] Decode batch. #running-req: 1, #token: 106, token usage: 0.01, gen throughput (token/s): 87.27, #queue-req: 0, 


[2025-03-19 07:18:18 TP0] Decode batch. #running-req: 1, #token: 146, token usage: 0.01, gen throughput (token/s): 95.45, #queue-req: 0, 


[2025-03-19 07:18:18 TP0] Decode batch. #running-req: 1, #token: 186, token usage: 0.01, gen throughput (token/s): 98.06, #queue-req: 0, 
[2025-03-19 07:18:18] INFO:     127.0.0.1:48254 - "POST /v1/chat/completions HTTP/1.1" 200 OK


Streaming mode is also supported.

In [4]:
stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

[2025-03-19 07:18:18] INFO:     127.0.0.1:48254 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-03-19 07:18:18 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0, 


It looks like you're preparing for a test. How can I assist you? Do you

 need help with

[2025-03-19 07:18:19 TP0] Decode batch. #running-req: 1, #token: 62, token usage: 0.00, gen throughput (token/s): 93.50, #queue-req: 0, 
 a particular subject, or would you like me to generate some practice questions?

## Completions

### Usage
Completions API is similar to Chat Completions API, but without the `messages` parameter or chat templates.

In [5]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

print_highlight(f"Response: {response}")

[2025-03-19 07:18:19 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-19 07:18:19 TP0] Decode batch. #running-req: 1, #token: 34, token usage: 0.00, gen throughput (token/s): 96.03, #queue-req: 0, 


[2025-03-19 07:18:19 TP0] Decode batch. #running-req: 0, #token: 0, token usage: 0.00, gen throughput (token/s): 104.81, #queue-req: 0, 
[2025-03-19 07:18:19] INFO:     127.0.0.1:48254 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [6]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

print_highlight(f"Response: {response}")

[2025-03-19 07:18:19 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-19 07:18:20 TP0] Decode batch. #running-req: 1, #token: 51, token usage: 0.00, gen throughput (token/s): 90.41, #queue-req: 0, 


[2025-03-19 07:18:20 TP0] Decode batch. #running-req: 1, #token: 91, token usage: 0.00, gen throughput (token/s): 102.69, #queue-req: 0, 


[2025-03-19 07:18:21 TP0] Decode batch. #running-req: 1, #token: 131, token usage: 0.01, gen throughput (token/s): 100.39, #queue-req: 0, 


[2025-03-19 07:18:21] INFO:     127.0.0.1:48254 - "POST /v1/completions HTTP/1.1" 200 OK


## Structured Outputs (JSON, Regex, EBNF)

For OpenAI compatible structed outputs API, refer to [Structured Outputs](https://docs.sglang.ai/backend/structured_outputs.html#OpenAI-Compatible-API) for more details.


## Batches

Batches API for chat completions and completions are also supported. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [7]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Batch job created with ID: {batch_response.id}")

[2025-03-19 07:18:21] INFO:     127.0.0.1:53410 - "POST /v1/files HTTP/1.1" 200 OK
[2025-03-19 07:18:21] INFO:     127.0.0.1:53410 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-03-19 07:18:21 TP0] Prefill batch. #new-seq: 2, #new-token: 20, #cached-token: 60, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [8]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        print_highlight(f"Request {result['custom_id']}:")
        print_highlight(f"Response: {result['response']}")

    print_highlight("Cleaning up files...")
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    print_highlight(f"Batch job failed with status: {batch_response.status}")
    if hasattr(batch_response, "errors"):
        print_highlight(f"Errors: {batch_response.errors}")

[2025-03-19 07:18:21 TP0] Decode batch. #running-req: 2, #token: 80, token usage: 0.00, gen throughput (token/s): 90.91, #queue-req: 0, 


Batch job status: validating...trying again in 3 seconds...
[2025-03-19 07:18:24] INFO:     127.0.0.1:53410 - "GET /v1/batches/batch_cabd1854-2858-4b7a-b52f-cdd4857ed6df HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2025-03-19 07:18:24] INFO:     127.0.0.1:53410 - "GET /v1/files/backend_result_file-a640a399-af64-4606-adca-499aee7c3c99/content HTTP/1.1" 200 OK


[2025-03-19 07:18:24] INFO:     127.0.0.1:53410 - "DELETE /v1/files/backend_result_file-a640a399-af64-4606-adca-499aee7c3c99 HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [9]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(20):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 64,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    print_highlight(
        f"Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}"
    )
    print_highlight(
        f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
    )

    time.sleep(3)

[2025-03-19 07:18:24] INFO:     127.0.0.1:53422 - "POST /v1/files HTTP/1.1" 200 OK
[2025-03-19 07:18:24] INFO:     127.0.0.1:53422 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-03-19 07:18:24 TP0] Prefill batch. #new-seq: 2, #new-token: 60, #cached-token: 50, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-19 07:18:24 TP0] Prefill batch. #new-seq: 18, #new-token: 540, #cached-token: 450, token usage: 0.00, #running-req: 2, #queue-req: 0, 
[2025-03-19 07:18:24 TP0] Decode batch. #running-req: 20, #token: 725, token usage: 0.04, gen throughput (token/s): 51.70, #queue-req: 0, 


[2025-03-19 07:18:25 TP0] Decode batch. #running-req: 20, #token: 1525, token usage: 0.07, gen throughput (token/s): 1901.07, #queue-req: 0, 


[2025-03-19 07:18:34] INFO:     127.0.0.1:34822 - "GET /v1/batches/batch_2173dc02-82c8-4fac-8a2c-69debd50a1ef HTTP/1.1" 200 OK


[2025-03-19 07:18:37] INFO:     127.0.0.1:34822 - "GET /v1/batches/batch_2173dc02-82c8-4fac-8a2c-69debd50a1ef HTTP/1.1" 200 OK


[2025-03-19 07:18:40] INFO:     127.0.0.1:34822 - "GET /v1/batches/batch_2173dc02-82c8-4fac-8a2c-69debd50a1ef HTTP/1.1" 200 OK


[2025-03-19 07:18:43] INFO:     127.0.0.1:34822 - "GET /v1/batches/batch_2173dc02-82c8-4fac-8a2c-69debd50a1ef HTTP/1.1" 200 OK


[2025-03-19 07:18:46] INFO:     127.0.0.1:34822 - "GET /v1/batches/batch_2173dc02-82c8-4fac-8a2c-69debd50a1ef HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [10]:
import json
import time
from openai import OpenAI
import os

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(5000):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 128,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    print_highlight(f"Cancellation initiated. Status: {cancelled_job.status}")
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        print_highlight(f"Current status: {cancelled_job.status}")

    # Verify final status
    assert cancelled_job.status == "cancelled"
    print_highlight("Batch job successfully cancelled")

except Exception as e:
    print_highlight(f"Error during cancellation: {e}")
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            print_highlight("Successfully cleaned up input file")
        if os.path.exists(input_file_path):
            os.remove(input_file_path)
            print_highlight("Successfully deleted local batch_requests.jsonl file")
    except Exception as e:
        print_highlight(f"Error cleaning up: {e}")
        raise e

[2025-03-19 07:18:49] INFO:     127.0.0.1:40348 - "POST /v1/files HTTP/1.1" 200 OK
[2025-03-19 07:18:49] INFO:     127.0.0.1:40348 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-03-19 07:18:50 TP0] Prefill batch. #new-seq: 16, #new-token: 16, #cached-token: 864, token usage: 0.02, #running-req: 0, #queue-req: 0, 
[2025-03-19 07:18:50 TP0] Prefill batch. #new-seq: 117, #new-token: 3394, #cached-token: 3041, token usage: 0.03, #running-req: 16, #queue-req: 281, 


[2025-03-19 07:18:51 TP0] Decode batch. #running-req: 133, #token: 6808, token usage: 0.33, gen throughput (token/s): 117.84, #queue-req: 4867, 


[2025-03-19 07:18:51 TP0] Decode batch. #running-req: 133, #token: 12128, token usage: 0.59, gen throughput (token/s): 11041.62, #queue-req: 4867, 


[2025-03-19 07:18:51 TP0] Decode batch. #running-req: 133, #token: 17448, token usage: 0.85, gen throughput (token/s): 11231.20, #queue-req: 4867, 


...

[2025-03-19 07:18:59 TP0] Decode batch. #running-req: 132, #token: 17895, token usage: 0.87, gen throughput (token/s): 11145.59, #queue-req: 4360, 


[2025-03-19 07:18:59 TP0] Prefill batch. #new-seq: 9, #new-token: 270, #cached-token: 225, token usage: 0.92, #running-req: 122, #queue-req: 4351, 
[2025-03-19 07:18:59] INFO:     127.0.0.1:51864 - "POST /v1/batches/batch_6dd7107b-428c-41b5-b556-3cbad82e5a18/cancel HTTP/1.1" 200 OK


[2025-03-19 07:18:59 TP0] Prefill batch. #new-seq: 15, #new-token: 2295, #cached-token: 375, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-19 07:19:02] INFO:     127.0.0.1:51864 - "GET /v1/batches/batch_6dd7107b-428c-41b5-b556-3cbad82e5a18 HTTP/1.1" 200 OK


[2025-03-19 07:19:02] INFO:     127.0.0.1:51864 - "DELETE /v1/files/backend_input_file-0a707d6b-34db-4a4d-983f-395f99319936 HTTP/1.1" 200 OK


In [11]:
terminate_process(server_process)